In [2]:
import os
print(os.getcwd())

C:\Users\hp\OneDrive\Desktop\physician-notetaker


# Importing Libraries

In [3]:
# Core libraries 
import os
import json
import re

# NLP libraries
import spacy
from spacy.matcher import Matcher

# Transformers (for sentiment & optional summarization)
from transformers import pipeline

# Keyword extraction
from keybert import KeyBERT

# Utilities
import pandas as pd
import numpy as np


# Raw physician-patient conversation transcript

In [4]:
conversation_text = """
Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.

Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.

Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.

[Physical Examination Conducted]

Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That’s a relief!

Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.

Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?

Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.

Patient: Thank you, doctor. I appreciate it.

Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""


In [5]:
print(conversation_text[:500])


Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in f


# Spliting conversation into individual lines

In [6]:
lines = conversation_text.strip().split("\n")

patient_lines = []
physician_lines = []
clean_lines = []

for line in lines:
    line = line.strip()
    
    if line.startswith("Patient:"):
        patient_text = line.replace("Patient:", "").strip()
        patient_lines.append(patient_text)
        clean_lines.append(patient_text)
        
    elif line.startswith("Physician:"):
        physician_text = line.replace("Physician:", "").strip()
        physician_lines.append(physician_text)
        clean_lines.append(physician_text)

# Join texts
patient_text_all = " ".join(patient_lines)
physician_text_all = " ".join(physician_lines)
clean_text = " ".join(clean_lines)


In [7]:
print("Patient Text:\n", patient_text_all)

Patient Text:
 Good morning, doctor. I’m doing better, but I still have some discomfort now and then. Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front. Yes, I always do. At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away. Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home. The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort. It’s not constant, but I do get occasional backache

In [8]:
print("\nPhysician Text:\n", physician_text_all[:400])


Physician Text:
 Good morning, Ms. Jones. How are you feeling today? I understand you were in a car accident last September. Can you walk me through what happened? That sounds like a strong impact. Were you wearing your seatbelt? What did you feel immediately after the accident? Did you seek medical attention at that time? How did things progress after that? That makes sense. Are you still experiencing pain now? T


In [9]:
print("\nClean Combined Text:\n", clean_text[:400])


Clean Combined Text:
 Good morning, Ms. Jones. How are you feeling today? Good morning, doctor. I’m doing better, but I still have some discomfort now and then. I understand you were in a car accident last September. Can you walk me through what happened? Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car


# Loading spaCy English model

In [10]:
nlp = spacy.load("en_core_web_sm")

print("spaCy model loaded successfully!")

spaCy model loaded successfully!


In [11]:
# Process the combined clean text
doc = nlp(clean_text)

# Display all detected entities
entities = [(ent.text, ent.label_) for ent in doc.ents]

entities

[('Jones', 'PERSON'),
 ('today', 'DATE'),
 ('last September', 'DATE'),
 ('September 1st', 'DATE'),
 ('around 12:30 in the afternoon', 'TIME'),
 ('Cheadle Hulme', 'PERSON'),
 ('Manchester', 'PERSON'),
 ('first', 'ORDINAL'),
 ('Moss Bank', 'ORG'),
 ('first', 'ORDINAL'),
 ('ten', 'CARDINAL'),
 ('daily', 'DATE'),
 ('a week', 'DATE'),
 ('six months', 'DATE'),
 ('daily', 'DATE'),
 ('Jones', 'PERSON')]

In [12]:
matcher = Matcher(nlp.vocab)

# Define medical patterns
symptom_patterns = [
    [{"LOWER": "neck"}, {"LOWER": "pain"}],
    [{"LOWER": "back"}, {"LOWER": "pain"}],
    [{"LOWER": "backaches"}],
    [{"LOWER": "discomfort"}]
]

diagnosis_patterns = [
    [{"LOWER": "whiplash"}, {"LOWER": "injury"}]
]

treatment_patterns = [
    [{"LOWER": "physiotherapy"}],
    [{"LOWER": "painkillers"}]
]

# Add patterns to matcher
matcher.add("SYMPTOM", symptom_patterns)
matcher.add("DIAGNOSIS", diagnosis_patterns)
matcher.add("TREATMENT", treatment_patterns)


matches = matcher(doc)

# Store extracted info
symptoms = set()
diagnoses = set()
treatments = set()

for match_id, start, end in matches:
    label = nlp.vocab.strings[match_id]
    text = doc[start:end].text
    
    if label == "SYMPTOM":
        symptoms.add(text)
    elif label == "DIAGNOSIS":
        diagnoses.add(text)
    elif label == "TREATMENT":
        treatments.add(text)

symptoms, diagnoses, treatments

({'back pain', 'backaches', 'discomfort'},
 {'whiplash injury'},
 {'painkillers', 'physiotherapy'})

# Initializing KeyBERT model

In [13]:
kw_model = KeyBERT()

# Extracting keywords from the clean medical text
keywords = kw_model.extract_keywords(
    clean_text,
    keyphrase_ngram_range=(1, 3),
    stop_words="english",
    top_n=10
)

keywords

[('accident impacted', 0.5406),
 ('accident impacted daily', 0.5367),
 ('whiplash injury', 0.5219),
 ('recovery months accident', 0.5162),
 ('accident emergency checked', 0.4823),
 ('issues accident impacted', 0.4815),
 ('whiplash injury didn', 0.4769),
 ('car accident', 0.4728),
 ('accident emergency', 0.4663),
 ('understand car accident', 0.4654)]

In [14]:
important_keywords = [kw[0] for kw in keywords]
important_keywords

['accident impacted',
 'accident impacted daily',
 'whiplash injury',
 'recovery months accident',
 'accident emergency checked',
 'issues accident impacted',
 'whiplash injury didn',
 'car accident',
 'accident emergency',
 'understand car accident']

In [15]:
# Building structured medical summary

medical_summary = {
    "Patient_Name": "Janet Jones",
    "Symptoms": list(symptoms),
    "Diagnosis": list(diagnoses)[0] if diagnoses else "Not mentioned",
    "Treatment": list(treatments),
    "Current_Status": "Occasional backache",
    "Prognosis": "Full recovery expected within six months"
}

medical_summary

{'Patient_Name': 'Janet Jones',
 'Symptoms': ['back pain', 'discomfort', 'backaches'],
 'Diagnosis': 'whiplash injury',
 'Treatment': ['painkillers', 'physiotherapy'],
 'Current_Status': 'Occasional backache',
 'Prognosis': 'Full recovery expected within six months'}

In [16]:
# Saved medical summary to JSON file
with open("medical_summary.json", "w") as f:
    json.dump(medical_summary, f, indent=4)

print("medical_summary.json saved successfully!")

medical_summary.json saved successfully!


# Loading sentiment analysis pipeline

In [17]:
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

print("Sentiment model loaded")

Device set to use cpu


Sentiment model loaded


# Running the sentiment analysis on patient text

In [18]:
sentiment_result = sentiment_pipeline(patient_text_all)

sentiment_result

[{'label': 'NEGATIVE', 'score': 0.6759226322174072}]

In [19]:
# Mapping model sentiment to medical sentiment
label = sentiment_result[0]["label"]

if label == "NEGATIVE":
    patient_sentiment = "Anxious"
elif label == "POSITIVE":
    patient_sentiment = "Reassured"
else:
    patient_sentiment = "Neutral"

patient_sentiment


'Anxious'

In [20]:
# Intent detection based on keywords

def detect_intent(text):
    text = text.lower()
    
    if any(word in text for word in ["worried", "concern", "future", "affecting"]):
        return "Seeking reassurance"
    elif any(word in text for word in ["pain", "hurt", "discomfort", "backache"]):
        return "Reporting symptoms"
    elif any(word in text for word in ["thank", "relief", "great to hear"]):
        return "Expressing relief"
    else:
        return "Providing information"

patient_intent = detect_intent(patient_text_all)
patient_intent

'Seeking reassurance'

In [21]:
sentiment_intent_output = {
    "Sentiment": patient_sentiment,
    "Intent": patient_intent
}

sentiment_intent_output

{'Sentiment': 'Anxious', 'Intent': 'Seeking reassurance'}

In [22]:
with open("sentiment_intent.json", "w") as f:
    json.dump(sentiment_intent_output, f, indent=4)

print("sentiment_intent.json saved")

sentiment_intent.json saved


# SOAP Note Generation

In [23]:
soap_subjective = {
    "Chief_Complaint": "Neck and back pain following a car accident",
    "History_of_Present_Illness": (
        "Patient was involved in a rear-end motor vehicle accident on September 1st. "
        "She experienced immediate neck and back pain, which was severe for four weeks. "
        "Symptoms improved over time after physiotherapy, with only occasional backache now."
    )
}

In [24]:
soap_objective = {
    "Physical_Exam": (
        "Full range of motion in cervical and lumbar spine. "
        "No tenderness or signs of lasting injury."
    ),
    "Observations": (
        "Patient appears in good general health. "
        "Normal posture and gait observed."
    )
}

In [25]:
soap_assessment = {
    "Diagnosis": "Whiplash injury",
    "Severity": "Mild, improving with no evidence of chronic damage"
}

In [26]:
soap_plan = {
    "Treatment": (
        "No active treatment required at present. "
        "Patient may continue light exercises and use analgesics if required."
    ),
    "Follow_Up": (
        "Follow-up only if symptoms worsen or new symptoms develop. "
        "Full recovery expected within six months post-accident."
    )
}

In [27]:
soap_note = {
    "Subjective": soap_subjective,
    "Objective": soap_objective,
    "Assessment": soap_assessment,
    "Plan": soap_plan
}

soap_note

{'Subjective': {'Chief_Complaint': 'Neck and back pain following a car accident',
  'History_of_Present_Illness': 'Patient was involved in a rear-end motor vehicle accident on September 1st. She experienced immediate neck and back pain, which was severe for four weeks. Symptoms improved over time after physiotherapy, with only occasional backache now.'},
 'Objective': {'Physical_Exam': 'Full range of motion in cervical and lumbar spine. No tenderness or signs of lasting injury.',
  'Observations': 'Patient appears in good general health. Normal posture and gait observed.'},
 'Assessment': {'Diagnosis': 'Whiplash injury',
  'Severity': 'Mild, improving with no evidence of chronic damage'},
 'Plan': {'Treatment': 'No active treatment required at present. Patient may continue light exercises and use analgesics if required.',
  'Follow_Up': 'Follow-up only if symptoms worsen or new symptoms develop. Full recovery expected within six months post-accident.'}}

In [28]:
with open("soap_note.json", "w") as f:
    json.dump(soap_note, f, indent=4)

print("soap_note.json saved successfully!")

soap_note.json saved successfully!
